# Image viewer

This notebook is for inspecting timelapse microscopy data, with associated sinhgle-cell labels and tracks, showing the infection of human macrophages with Mycobacterium Tuberculosis (Mtb), acquired on an Opera Phenix confocal microscope. 

In [1]:
import napari
from octopuslite import utils, tile

### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [2]:
image_dir = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Images/'
metadata_fn = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Index.idx.xml'
metadata = utils.read_harmony_metadata(metadata_fn)

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

Extracting metadata complete!


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [3]:
metadata_path = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml'
assay_layout = utils.read_harmony_metadata(metadata_path, assay_layout=True, mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

Reading metadata XML file...


/home/dayn/analysis/octopuslite-reader/octopuslite/utils.py:322: FutureWarning: The default value of regex will change from True to False in a future version.
  corresponding_mask_fns = input_img_fns.str.replace('ch(\d+)', 'ch99')
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:948: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  element = np.asarray(element)


Extracting metadata complete!


Strain Compound Concentration ConcentrationEC  \
3 4    RD1     CTRL             0             EC0   
  5     WT     CTRL             0             EC0   
  6     WT      PZA            60            EC50   
  7     WT      RIF           0.1            EC50   
  8     WT      INH          0.04            EC50   
  9     WT      BDQ          0.02            EC50   
4 4    RD1     CTRL             0             EC0   
  5     WT     CTRL             0             EC0   
  6     WT      PZA            60            EC50   
  7     WT      RIF           0.1            EC50   
  8     WT      INH          0.04            EC50   
  9     WT      BDQ          0.02            EC50   
5 4    RD1     CTRL             0             EC0   
  5     WT     CTRL             0             EC0   
  6     WT      PZA           400            EC99   
  7     WT      RIF             2            EC99   
  8     WT      INH             2            EC99   
  9     WT      BDQ           2.5            EC99   
6 4    RD1     CTRL             0             EC0   
  5     WT     CTRL             0             EC0   
  6     WT      PZA           400            EC99   
  7     WT      RIF             2            EC99   
  8     WT      INH             2            EC99   
  9     WT      BDQ           2.5            EC99   

                                         Missing masks  
3 4                                               None  
  5  (2025, [/mnt/DATA/sandbox/pierre_live_cell_dat...  
  6  (2025, [/mnt/DATA/sandbox/pierre_live_cell_dat...  
  7                                               None  
  8  (1098, [/mnt/DATA/sandbox/pierre_live_cell_dat...  
  9  (2025, [/mnt/DATA/sandbox/pierre_live_cell_dat...  
4 4  (2025, [/mnt/DATA/sandbox/pierre_live_cell_dat...  
  5  (2025, [/mnt/DATA/sandbox/pierre_live_cell_dat...  
  6                                               None  
  7  (2025, [/mnt/DATA/sandbox/pierre_live_cell_dat...  
  8  (2025, [/mnt/DATA/sandbox/pierre_live_cell_dat...  
  9                                               None  
5 4                                               None  
  5                                               None  
  6  (2025, [/mnt/DATA/sandbox/pierre_live_cell_dat...  
  7  (225, [/mnt/DATA/sandbox/pierre_live_cell_data...  
  8                                               None  
  9  (2025, [/mnt/DATA/sandbox/pierre_live_cell_dat...  
6 4  (2025, [/mnt/DATA/sandbox/pierre_live_cell_dat...  
  5  (2025, [/mnt/DATA/sandbox/pierre_live_cell_dat...  
  6                                               None  
  7  (2025, [/mnt/DATA/sandbox/pierre_live_cell_dat...  
  8  (2025, [/mnt/DATA/sandbox/pierre_live_cell_dat...  
  9                                               None

#### Iterate over all positions and create animations

In [4]:
from napari_animation import Animation
from tqdm.auto import tqdm

In [ ]:
for row in tqdm(assay_layout.iterrows(), total = len(assay_layout)):
    strain = row[1]['Strain']
    compound = row[1]['Compound']
    conc = row[1]['Concentration']
    name = f'{strain, compound, conc}'
    
    row, column = row[0]
    images = tile.compile_mosaic(image_dir, metadata, row, column,set_plane=1)
    
    viewer = napari.Viewer()
    viewer.add_image(images, 
                 channel_axis=1,
                 name=[f"macrophage {name}", "mtb"],
                 colormap=["green", "magenta"],
                 contrast_limits=[[100, 2000], [100, 500]]
                 )
    
    animation = Animation(viewer)
    viewer.update_console({'animation': animation})
    viewer.camera.center = (0, 0, 3024, 3024)
    viewer.dims.current_step = (0, 0, 0, 0)
    viewer.camera.zoom = 0.109
    animation.capture_keyframe(steps = 100)
    viewer.dims.current_step = (74, 0, 0, 0)
    animation.capture_keyframe(steps = 100)
    fn = f'/home/dayn/Videos/tb_mp4s/tiling/tiling{row,column,strain,compound,conc}.mp4'
    animation.animate(fn, 
                      canvas_only=True,
                      fps = 7,
                      quality = 9)
    
    

  0%|          | 0/24 [00:00<?, ?it/s]

Assistant skips harvesting pyclesperanto as it's not installed.


/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


Rendering frames...



  0%|                                                                          | 0/101 [00:00<?, ?it/s]12-Jan-23 11:33:18 - imageio_ffmpeg - WARNING  - IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1124, 699) to (1136, 704) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).

 96%|██████████████████████████████████████████████████████████████▍  | 97/101 [09:11<00:25,  6.43s/it]


...


  0%|                                                                          | 0/101 [00:00<?, ?it/s]12-Jan-23 13:03:24 - imageio_ffmpeg - WARNING  - IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1124, 699) to (1136, 704) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).

 96%|██████████████████████████████████████████████████████████████▍  | 97/101 [10:36<00:27,  6.84s/it]


100%|████████████████████████████████████████████████████████████████| 101/101 [11:00<00:00,  6.54s/it]
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


Rendering frames...



  0%|                                                                          | 0/101 [00:00<?, ?it/s]12-Jan-23 13:14:52 - imageio_ffmpeg - WARNING  - IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1124, 699) to (1136, 704) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).

 96%|██████████████████████████████████████████████████████████████▍  | 97/101 [09:40<00:28,  7.01s/it]


100%|████████████████████████████████████████████████████████████████| 101/101 [10:05<00:00,  5.99s/it]
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


In [ ]:
viewer = napari.Viewer()
    viewer.add_image(images, 
                 channel_axis=1,
                 name=[f"macrophage {name}", "mtb"],
                 colormap=["green", "magenta"],
                 contrast_limits=[[100, 2000], [100, 500]]
                 )
    
    animation = Animation(viewer)
    viewer.update_console({'animation': animation})
    viewer.camera.center = (0, 0, 3024, 3024)
    viewer.dims.current_step = (0, 0, 0, 0)
    viewer.camera.zoom=0.13535
    animation.capture_keyframe(steps = 100)
    viewer.dims.current_step = (74, 0, 0, 0)
    animation.capture_keyframe(steps = 100)
    fn = f'/home/dayn/Videos/tb_mp4s/tiling/tiling_slight_zoom_{row,column,strain,compound,conc}.mp4'
    animation.animate(fn, 
                      canvas_only=True,
                      fps = 7,
                      quality = 9)
    
    viewer = napari.Viewer()
    viewer.add_image(images, 
                 channel_axis=1,
                 name=[f"macrophage {name}", "mtb"],
                 colormap=["green", "magenta"],
                 contrast_limits=[[100, 2000], [100, 500]]
                 )
    
    animation = Animation(viewer)
    viewer.update_console({'animation': animation})
    viewer.camera.center = (0, 0, 2164, 3994)
    viewer.dims.current_step = (0, 0, 0, 0)
    viewer.camera.zoom = 0.962
    animation.capture_keyframe(steps = 100)
    viewer.dims.current_step = (74, 0, 0, 0)
    animation.capture_keyframe(steps = 100)
    fn = f'/home/dayn/Videos/tb_mp4s/tiling/zoom_right_tiling{row,column,strain,compound,conc}.mp4'
    animation.animate(fn, 
                      canvas_only=True,
                      fps = 7,
                      quality = 9)

    viewer = napari.Viewer()
    viewer.add_image(images, 
                 channel_axis=1,
                 name=[f"macrophage {name}", "mtb"],
                 colormap=["green", "magenta"],
                 contrast_limits=[[100, 2000], [100, 500]]
                 )
    
    animation = Animation(viewer)
    viewer.update_console({'animation': animation})
    viewer.camera.center = (0, 0, 3122, 2976)
    viewer.dims.current_step = (0, 0, 0, 0)
    viewer.camera.zoom = 0.4169
    animation.capture_keyframe(steps = 100)
    viewer.dims.current_step = (74, 0, 0, 0)
    animation.capture_keyframe(steps = 100)
    fn = f'/home/dayn/Videos/tb_mp4s/tiling/zoom_center_tiling{row,column,strain,compound,conc}.mp4'
    animation.animate(fn, 
                      canvas_only=True,
                      fps = 7,
                      quality = 9)

# Tests

In [35]:
viewer = napari.Viewer()
viewer.add_image(images, 
             channel_axis=1,
             name=[f"macrophage {name}", "mtb"],
             colormap=["green", "magenta"],
             contrast_limits=[[100, 2000], [100, 500]]
             )

animation = Animation(viewer)
viewer.update_console({'animation': animation})
viewer.camera.center = (0, 0, 3024, 3024)
viewer.camera.zoom = 0.18
animation.capture_keyframe(steps = 100)
viewer.dims.current_step = (74, 0, 0, 0)
animation.capture_keyframe(steps = 100)
fn = f'/home/dayn/Videos/tb_mp4s/tiling/tiling{row,column,strain,compound,conc}.mp4'
animation.animate(fn, 
                  canvas_only=True,
                     fps = 7,
                     quality = 9)

animation = Animation(viewer)
viewer.update_console({'animation': animation})
viewer.camera.center = (0, 0, 2164, 3994)
viewer.camera.zoom = 0.962
animation.capture_keyframe(steps = 100)
viewer.dims.current_step = (74, 0, 0, 0)
animation.capture_keyframe(steps = 100)
fn = f'/home/dayn/Videos/tb_mp4s/tiling/zoom_tiling{row,column,strain,compound,conc}.mp4'
animation.animate(fn, 
                  canvas_only=True,
                     fps = 7,
                     quality = 9)

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


Rendering frames...


100%|████████████████████████████████████████████████████████████████| 101/101 [04:46<00:00,  2.83s/it]


In [50]:
viewer = napari.Viewer()
viewer.add_image(images, 
             channel_axis=1,
             name=[f"macrophage {name}", "mtb"],
             colormap=["green", "magenta"],
             contrast_limits=[[100, 2000], [100, 500]]
             )

# animation = Animation(viewer)
# viewer.update_console({'animation': animation})
# viewer.camera.center = (0, 0, 3024, 3024)
# viewer.camera.zoom = 0.5
# animation.capture_keyframe(steps = 10)
# viewer.dims.current_step = (10, 0, 0, 0)
# animation.capture_keyframe(steps = 10)
# fn = f'/home/dayn/Videos/tb_mp4s/tiling/tiling{row,column,strain,compound,conc}.mp4'
# animation.animate(fn, 
#                   canvas_only=True,
#                      fps = 7,
#                      quality = 9)

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


[<Image layer "macrophage ('WT', 'CTRL', '0')" at 0x7fd50f86c040>,
 <Image layer 'mtb' at 0x7fd519c9d850>]

In [52]:
viewer.camera

Camera(center=(0.0, 2964.168687360873, 2992.54454118942), zoom=0.13535462972612866, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True)